# Exercise 1
Author: Tobias Famos
## Loading the Data

In [200]:
import pandas
df = pandas.read_csv("Data/titanic.csv")
df.head(10)

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
5,0,3,Mr. James Moran,male,27.0,0,0,8.4583
6,0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
7,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750
8,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson,female,27.0,0,2,11.1333
9,1,2,Mrs. Nicholas (Adele Achem) Nasser,female,14.0,1,0,30.0708


# Task 1
First lets define a function to evaluate prediction rules. Prediction rules will be give and a lambda function doing it's decision based only on a single row.

In [201]:
def is_correct_prediction(row, prediction_name):
    if row['Survived'] == row[prediction_name]:
        return True
    else:
        return False

def predict_and_evaluate(func):
    df['prediction'] = df.apply (lambda row: func(row), axis=1)
    correct_predictions = df.apply(lambda row: is_correct_prediction(row, 'prediction'), axis=1)
    accuracy =  correct_predictions.value_counts()[True] / correct_predictions.size
    print(f"Accuracy for {func.__name__}: {accuracy}")
    return accuracy


## Task 1.a) What is the best default rule for this dataset?
There are two possible rules without any prior knowledge of the person:
- survived = yes
- survived = no;
Let's count the values and see which brings the higher accuracy.

In [202]:
def label_all_died(row):
    return 0

def label_all_survived(row):
    return 1

predict_and_evaluate(label_all_died)
predict_and_evaluate(label_all_survived)



Accuracy for label_all_died: 0.6144306651634723
Accuracy for label_all_survived: 0.3855693348365276


0.3855693348365276

As one can see from the output above, the best accuracy is when we use the default rule survived = 0.

## Task 1.b)
What is the best 1R for this dataset?
Lets first import our OneRClassifier

In [203]:
from sklearn.model_selection import train_test_split
from mlxtend.evaluate import accuracy_score
from mlxtend.classifier import OneRClassifier


Define a function to evaluate a OneR based on one attribute

In [204]:
def create_and_evaluate_1R_for_attribute(attribute):
    x_d = df[[attribute]]
    y = df["Survived"]
    x_d_train, x_d_test, y_train, y_test = train_test_split(x_d, y, test_size=0.2)
    one_r = OneRClassifier()
    one_r.fit(x_d_train.to_numpy(), y_train)
    y_pred = one_r.predict(x_d_test.to_numpy())
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy for 1R based on attribute {attribute}: {accuracy}')

Evaluate all the possible Attributes

In [205]:
create_and_evaluate_1R_for_attribute('Pclass')
create_and_evaluate_1R_for_attribute('Sex')
create_and_evaluate_1R_for_attribute('Age')
create_and_evaluate_1R_for_attribute('Siblings/Spouses Aboard')
create_and_evaluate_1R_for_attribute('Parents/Children Aboard')
create_and_evaluate_1R_for_attribute('Fare')


Accuracy for 1R based on attribute Pclass: 0.702247191011236
Accuracy for 1R based on attribute Sex: 0.7696629213483146
Accuracy for 1R based on attribute Age: 0.6067415730337079
Accuracy for 1R based on attribute Siblings/Spouses Aboard: 0.6179775280898876
Accuracy for 1R based on attribute Parents/Children Aboard: 0.5955056179775281
Accuracy for 1R based on attribute Fare: 0.6910112359550562


So the best 1R Rule for this datasets seems to be based on sex. Let's write the predictor just for fun and see the error throughout the whole dataset.

In [206]:
def one_rule_sex(row):
    if row['Sex'] == 'female':
        return 1
    return 0
predict_and_evaluate(one_rule_sex)

Accuracy for one_rule_sex: 0.7857948139797069


0.7857948139797069

## Task 1.c) Can you produce a second rule based on a single attribute with a good effectiveness?

Sure, we will just take the second highest ranking rule above.

In [207]:
create_and_evaluate_1R_for_attribute('Pclass')
create_and_evaluate_1R_for_attribute('Sex')
create_and_evaluate_1R_for_attribute('Age')
create_and_evaluate_1R_for_attribute('Siblings/Spouses Aboard')
create_and_evaluate_1R_for_attribute('Parents/Children Aboard')
create_and_evaluate_1R_for_attribute('Fare')

Accuracy for 1R based on attribute Pclass: 0.6797752808988764
Accuracy for 1R based on attribute Sex: 0.7752808988764045
Accuracy for 1R based on attribute Age: 0.5898876404494382
Accuracy for 1R based on attribute Siblings/Spouses Aboard: 0.6067415730337079
Accuracy for 1R based on attribute Parents/Children Aboard: 0.6292134831460674
Accuracy for 1R based on attribute Fare: 0.6685393258426966



Depending on the Test / Train split we run here, this is either the second best attribute to pick is either Fare or Class. As the Fare is just dependent on the class (First class being more expensive than third, we will just pick the class as the second rule).
Once again, I write the rule function and evaluate it over the whole dataset.

In [208]:
def one_rule_class(row):
    if row['Pclass'] == 1:
        return 1
    return 0

predict_and_evaluate(one_rule_class)

Accuracy for one_rule_class: 0.677564825253664


0.677564825253664